In [2]:
# Dependencies
import pandas as pd
import requests
import gmaps
from pprint import pprint
import numpy as np

from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [3]:
covid_df = pd.read_csv('Resources/Covid_new_data.csv')
covid_df
grouped_df = covid_df.groupby("Country").sum()
grouped_df


,Unnamed: 0,Total Cases,Total Deaths,Total Recovered
Country,,,,
Afghanistan,15753,1578381,36980,555604
Albania,47437,152469,4468,93895
Algeria,79121,927544,66804,584334
Andorra,110805,85500,4903,61044
Angola,142489,14401,740,4276
...,...,...,...,...
Viet Nam,111257209,36622,0,29209
Western Sahara,111288893,801,53,568
Yemen,111320577,50731,13092,16209


In [4]:
grouped_df.reset_index(inplace=True)
grouped_df

,Country,Unnamed: 0,Total Cases,Total Deaths,Total Recovered
0,Afghanistan,15753,1578381,36980,555604
1,Albania,47437,152469,4468,93895
2,Algeria,79121,927544,66804,584334
3,Andorra,110805,85500,4903,61044
4,Angola,142489,14401,740,4276
...,...,...,...,...,...
181,Viet Nam,111257209,36622,0,29209
182,Western Sahara,111288893,801,53,568
183,Yemen,111320577,50731,13092,16209
184,Zambia,111352261,92160,1331,65298


In [5]:
params = {"key": gkey}


for i,row in grouped_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    country = row['Country']
    params['address'] = f"{country}"
    try:
        response = requests.get(base_url, params=params).json()    
        grouped_df.loc[i, 'Lat'] = response["results"][0]["geometry"]["location"]["lat"]
        grouped_df.loc[i, 'Lng'] = response["results"][0]["geometry"]["location"]["lng"]

    except:
        print(f"{country} not found. skipping....")
        pass

grouped_df.head(20)

,Country,Unnamed: 0,Total Cases,Total Deaths,Total Recovered,Lat,Lng
0,Afghanistan,15753,1578381,36980,555604,33.939110,67.709953
1,Albania,47437,152469,4468,93895,41.153332,20.168331
2,Algeria,79121,927544,66804,584334,28.033886,1.659626
3,Andorra,110805,85500,4903,61044,42.506285,1.521801
4,Angola,142489,14401,740,4276,-11.202692,17.873887
5,Antigua and Barbuda,174173,3699,296,2011,17.060816,-61.796428
6,Argentina,205857,3002764,71418,1058893,-38.416097,-63.616672
7,Armenia,237541,1227428,20331,608227,40.069099,45.038189
8,Australia,3040952,825566,10039,624329,-25.274398,133.775136
9,Austria,522697,1834578,64277,1459492,47.516231,14.550072


In [22]:
locations = grouped_df[["Lat", "Lng"]]

rating = grouped_df["Total Cases"]
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))